In [1]:
import torch
import torch.nn as nn
from torch.optim import SGD
import math
import numpy as np

In [2]:
torch.manual_seed(1017)

In [3]:
def mkDataSet(data_size, data_length=50, freq=60., noise=0.02):
    """
    params
      data_size : データセットサイズ
      data_length : 各データの時系列長
      freq : 周波数
      noise : ノイズの振幅
    returns
      train_x : トレーニングデータ（t=1,2,...,size-1の値)
      train_t : トレーニングデータのラベル（t=sizeの値）
    """
    train_x = []
    train_t = []

    for offset in range(data_size):
        train_x.append([[math.sin(2 * math.pi * (offset + i) / freq) + np.random.normal(loc=0.0, scale=noise)] for i in range(data_length)])
        train_t.append([math.sin(2 * math.pi * (offset + data_length) / freq)])

    return train_x, train_t

In [4]:
class Predictor(nn.Module):
    def __init__(self, inputDim, hiddenDim, outputDim):
        super(Predictor, self).__init__()

        self.rnn = nn.LSTM(input_size = inputDim,
                            hidden_size = hiddenDim,
                            batch_first = True)
        self.output_layer = nn.Linear(hiddenDim, outputDim)
    
    def forward(self, inputs, hidden0=None):
        output, (hidden, cell) = self.rnn(inputs, hidden0) #LSTM層
        output = self.output_layer(output[:, -1, :]) #全結合層

        return output

In [5]:
training_size = 100 #traning dataのデータ数
epochs_num = 1000 #traningのepoch回数
hidden_size = 5 #LSTMの隠れ層の次元数

train_x, train_t = mkDataSet(training_size) #Datasetの作成

model = Predictor(1, hidden_size, 1) #modelの宣言

criterion = nn.MSELoss() #評価関数の宣言
optimizer = SGD(model.parameters(), lr=0.01) #最適化関数の宣言}

In [6]:
train_x[0]

[[0.016904171041831777],
 [0.11425490580939804],
 [0.23673697509116806],
 [0.30678324559886216],
 [0.42170992902598614],
 [0.4849151372922008],
 [0.6138813994044291],
 [0.6647565036512278],
 [0.7385089633880186],
 [0.7714483018814303],
 [0.8643916805161332],
 [0.8951319238568471],
 [0.9585758684302778],
 [0.9495062886244593],
 [0.9682634745332497],
 [1.0168653912399674],
 [0.9815840582171123],
 [0.9640332460312924],
 [0.910831601004271],
 [0.913327372387491],
 [0.908306035412673],
 [0.8342649530419809],
 [0.7372705494503413],
 [0.6700462548532169],
 [0.5573641189928182],
 [0.5077702900187906],
 [0.42199797548827894],
 [0.3016608674221748],
 [0.21229314515366754],
 [0.12998816854901216],
 [-0.004138231129648321],
 [-0.06840103788146149],
 [-0.22023315149847753],
 [-0.30239089047376383],
 [-0.4116324729639558],
 [-0.5305192403891544],
 [-0.5791263287136266],
 [-0.679880543915686],
 [-0.7661426527211158],
 [-0.8187674699087375],
 [-0.8776936363888214],
 [-0.906060172101305],
 [-0.95250392

In [24]:
for epoch in range(epochs_num):
    # training
    running_loss = 0.0
    training_accuracy = 0.0
    for i in range(training_size):
        optimizer.zero_grad()
        data = torch.tensor([train_x[i]])
        label = torch.tensor([train_t[i]])

        output = model(data)

        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        running_loss += loss.data.item()
        training_accuracy += np.sum(np.abs((output.data - label.data).numpy()) < 0.1) #outputとlabelの誤差が0.1以内なら正しいとみなす。
    training_accuracy /= training_size
    print('%d loss: %.3f, training_accuracy: %.5f' % (epoch + 1, running_loss, training_accuracy))

1 loss: 33.072, training_accuracy: 0.09000
2 loss: 20.486, training_accuracy: 0.11000
3 loss: 10.432, training_accuracy: 0.14000
4 loss: 6.549, training_accuracy: 0.18000
5 loss: 5.232, training_accuracy: 0.19000
6 loss: 4.595, training_accuracy: 0.19000
7 loss: 4.163, training_accuracy: 0.19000
8 loss: 3.816, training_accuracy: 0.18000
9 loss: 3.516, training_accuracy: 0.20000
10 loss: 3.247, training_accuracy: 0.23000
11 loss: 3.001, training_accuracy: 0.26000
12 loss: 2.773, training_accuracy: 0.26000
13 loss: 2.559, training_accuracy: 0.28000
14 loss: 2.356, training_accuracy: 0.31000
15 loss: 2.163, training_accuracy: 0.33000
16 loss: 1.977, training_accuracy: 0.34000
17 loss: 1.798, training_accuracy: 0.35000
18 loss: 1.626, training_accuracy: 0.37000
19 loss: 1.461, training_accuracy: 0.40000
20 loss: 1.303, training_accuracy: 0.43000
21 loss: 1.153, training_accuracy: 0.48000
22 loss: 1.014, training_accuracy: 0.50000
23 loss: 0.888, training_accuracy: 0.55000
24 loss: 0.775, t

KeyboardInterrupt: 

In [14]:
def mkRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す。
    """
    batch_x = []
    batch_t = []

    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - 1)
        batch_x.append(train_x[idx])
        batch_t.append(train_t[idx])
    
    return torch.tensor(batch_x), torch.tensor(batch_t)

In [25]:
batch_size = 10

for epoch in range(epochs_num):
    # training
    running_loss = 0.0
    training_accuracy = 0.0
    for i in range(int(training_size / batch_size)):
        optimizer.zero_grad()

        data, label = mkRandomBatch(train_x, train_t, batch_size)

        output = model(data)

        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        running_loss += loss.data.item()
        training_accuracy += np.sum(np.abs((output.data - label.data).numpy()) < 0.1) #outputとlabelの誤差が0.1以内なら正しいとみなす。
    training_accuracy /= training_size
    print('%d loss: %.3f, training_accuracy: %.5f' % (epoch + 1, running_loss, training_accuracy))

1 loss: 0.043, training_accuracy: 0.81000
2 loss: 0.044, training_accuracy: 0.90000
3 loss: 0.041, training_accuracy: 0.86000
4 loss: 0.035, training_accuracy: 0.95000
5 loss: 0.035, training_accuracy: 0.94000
6 loss: 0.036, training_accuracy: 0.94000
7 loss: 0.032, training_accuracy: 0.93000
8 loss: 0.033, training_accuracy: 0.96000
9 loss: 0.031, training_accuracy: 0.96000
10 loss: 0.036, training_accuracy: 0.94000
11 loss: 0.032, training_accuracy: 0.96000
12 loss: 0.035, training_accuracy: 0.92000
13 loss: 0.030, training_accuracy: 0.98000
14 loss: 0.031, training_accuracy: 0.96000
15 loss: 0.028, training_accuracy: 0.97000
16 loss: 0.033, training_accuracy: 0.91000
17 loss: 0.031, training_accuracy: 0.97000
18 loss: 0.031, training_accuracy: 0.95000
19 loss: 0.030, training_accuracy: 0.97000
20 loss: 0.027, training_accuracy: 0.96000
21 loss: 0.028, training_accuracy: 0.97000
22 loss: 0.028, training_accuracy: 0.95000
23 loss: 0.031, training_accuracy: 0.95000
24 loss: 0.028, trai

191 loss: 0.022, training_accuracy: 0.98000
192 loss: 0.018, training_accuracy: 0.98000
193 loss: 0.022, training_accuracy: 0.98000
194 loss: 0.019, training_accuracy: 0.98000
195 loss: 0.024, training_accuracy: 0.94000
196 loss: 0.022, training_accuracy: 0.97000
197 loss: 0.020, training_accuracy: 0.98000
198 loss: 0.025, training_accuracy: 0.99000
199 loss: 0.020, training_accuracy: 1.00000
200 loss: 0.023, training_accuracy: 0.98000
201 loss: 0.024, training_accuracy: 1.00000
202 loss: 0.020, training_accuracy: 0.99000
203 loss: 0.016, training_accuracy: 1.00000
204 loss: 0.021, training_accuracy: 0.99000
205 loss: 0.022, training_accuracy: 0.98000
206 loss: 0.019, training_accuracy: 0.99000
207 loss: 0.019, training_accuracy: 0.99000
208 loss: 0.019, training_accuracy: 0.98000
209 loss: 0.019, training_accuracy: 0.98000
210 loss: 0.018, training_accuracy: 1.00000
211 loss: 0.021, training_accuracy: 0.96000
212 loss: 0.021, training_accuracy: 0.97000
213 loss: 0.021, training_accura

379 loss: 0.014, training_accuracy: 1.00000
380 loss: 0.015, training_accuracy: 1.00000
381 loss: 0.014, training_accuracy: 0.99000
382 loss: 0.016, training_accuracy: 0.99000
383 loss: 0.011, training_accuracy: 1.00000
384 loss: 0.013, training_accuracy: 1.00000
385 loss: 0.017, training_accuracy: 1.00000
386 loss: 0.014, training_accuracy: 1.00000
387 loss: 0.019, training_accuracy: 0.97000
388 loss: 0.017, training_accuracy: 0.99000
389 loss: 0.013, training_accuracy: 0.98000
390 loss: 0.014, training_accuracy: 0.98000
391 loss: 0.017, training_accuracy: 1.00000
392 loss: 0.017, training_accuracy: 0.99000
393 loss: 0.021, training_accuracy: 0.97000
394 loss: 0.016, training_accuracy: 1.00000
395 loss: 0.016, training_accuracy: 1.00000
396 loss: 0.014, training_accuracy: 0.99000
397 loss: 0.014, training_accuracy: 1.00000
398 loss: 0.016, training_accuracy: 1.00000
399 loss: 0.015, training_accuracy: 1.00000
400 loss: 0.014, training_accuracy: 1.00000
401 loss: 0.016, training_accura

567 loss: 0.013, training_accuracy: 1.00000
568 loss: 0.013, training_accuracy: 1.00000
569 loss: 0.014, training_accuracy: 1.00000
570 loss: 0.013, training_accuracy: 1.00000
571 loss: 0.012, training_accuracy: 1.00000
572 loss: 0.014, training_accuracy: 1.00000
573 loss: 0.012, training_accuracy: 1.00000
574 loss: 0.014, training_accuracy: 1.00000
575 loss: 0.011, training_accuracy: 1.00000
576 loss: 0.011, training_accuracy: 1.00000
577 loss: 0.012, training_accuracy: 1.00000
578 loss: 0.010, training_accuracy: 1.00000
579 loss: 0.012, training_accuracy: 1.00000
580 loss: 0.012, training_accuracy: 1.00000
581 loss: 0.013, training_accuracy: 1.00000
582 loss: 0.013, training_accuracy: 1.00000
583 loss: 0.013, training_accuracy: 1.00000
584 loss: 0.014, training_accuracy: 1.00000
585 loss: 0.011, training_accuracy: 1.00000
586 loss: 0.010, training_accuracy: 1.00000
587 loss: 0.014, training_accuracy: 1.00000
588 loss: 0.012, training_accuracy: 1.00000
589 loss: 0.013, training_accura

754 loss: 0.010, training_accuracy: 1.00000
755 loss: 0.010, training_accuracy: 1.00000
756 loss: 0.012, training_accuracy: 1.00000
757 loss: 0.008, training_accuracy: 1.00000
758 loss: 0.012, training_accuracy: 1.00000
759 loss: 0.010, training_accuracy: 1.00000
760 loss: 0.010, training_accuracy: 1.00000
761 loss: 0.010, training_accuracy: 1.00000
762 loss: 0.011, training_accuracy: 1.00000
763 loss: 0.010, training_accuracy: 1.00000
764 loss: 0.009, training_accuracy: 1.00000
765 loss: 0.009, training_accuracy: 1.00000
766 loss: 0.011, training_accuracy: 1.00000
767 loss: 0.009, training_accuracy: 1.00000
768 loss: 0.009, training_accuracy: 1.00000
769 loss: 0.009, training_accuracy: 1.00000
770 loss: 0.008, training_accuracy: 1.00000
771 loss: 0.012, training_accuracy: 1.00000
772 loss: 0.010, training_accuracy: 1.00000
773 loss: 0.009, training_accuracy: 1.00000
774 loss: 0.010, training_accuracy: 1.00000
775 loss: 0.007, training_accuracy: 1.00000
776 loss: 0.013, training_accura

942 loss: 0.009, training_accuracy: 1.00000
943 loss: 0.009, training_accuracy: 1.00000
944 loss: 0.008, training_accuracy: 1.00000
945 loss: 0.007, training_accuracy: 1.00000
946 loss: 0.010, training_accuracy: 1.00000
947 loss: 0.009, training_accuracy: 1.00000
948 loss: 0.007, training_accuracy: 1.00000
949 loss: 0.007, training_accuracy: 1.00000
950 loss: 0.008, training_accuracy: 1.00000
951 loss: 0.010, training_accuracy: 1.00000
952 loss: 0.010, training_accuracy: 1.00000
953 loss: 0.007, training_accuracy: 1.00000
954 loss: 0.009, training_accuracy: 1.00000
955 loss: 0.009, training_accuracy: 1.00000
956 loss: 0.009, training_accuracy: 1.00000
957 loss: 0.009, training_accuracy: 1.00000
958 loss: 0.008, training_accuracy: 1.00000
959 loss: 0.007, training_accuracy: 1.00000
960 loss: 0.007, training_accuracy: 1.00000
961 loss: 0.009, training_accuracy: 1.00000
962 loss: 0.008, training_accuracy: 1.00000
963 loss: 0.010, training_accuracy: 1.00000
964 loss: 0.007, training_accura

In [27]:
#test
test_accuracy = 0.0
test_size = 1000
test_x, test_t = mkDataSet(test_size)

for i in range(int(test_size / batch_size)):
    offset = i * batch_size
    data, label = torch.tensor(test_x[offset:offset+batch_size]), torch.tensor(test_t[offset:offset+batch_size])
    output = model(data, None)

    test_accuracy += np.sum(np.abs((output.data - label.data).numpy()) < 0.1)

training_accuracy /= training_size
test_accuracy /= test_size

print('%d loss: %.3f, training_accuracy: %.5f, test_accuracy: %.5f' % (
    epoch + 1, running_loss, training_accuracy, test_accuracy))

1000 loss: 0.008, training_accuracy: 0.01000, test_accuracy: 1.00000
